# VERTEX AI


In [25]:
# pip install --upgrade google-cloud-aiplatform
# gcloud auth application-default login

In [11]:
#!gcloud auth application-default login

^C


In [1]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, ChatSession
import vertexai.preview.generative_models as generative_models
import pprint

In [2]:
from dotenv import dotenv_values

config = dotenv_values(".env")

In [3]:
vertexai.init(project=config['PROJECT'], location=config['REGION'])

In [4]:
def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

In [5]:
model = GenerativeModel(
    "gemini-1.5-pro-preview-0409",
    system_instruction=["""You a helpful agent who helps to extract relevant information from documents"""],
    safety_settings= {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
},
     generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.1,
    "top_p": 0.95,
}
  )
chat = model.start_chat(response_validation=False)

In [6]:
from pathlib import Path
content=[]
pathlist = Path("data/project/Dolores_Reyes_IPA/Informes").glob('**/*.pdf')
for path in pathlist:
    # Convert path object to a string
    path_in_str = str(path)
    # Perform actions on the file (e.g., print its path)
    print(path_in_str)
    with open(path_in_str, "rb") as f:
        im_bytes = f.read()
        im_b64 = base64.b64encode(im_bytes).decode("utf8")
        image = Part.from_data(data=im_b64, mime_type="application/pdf")
        content.append(image)

data\project\Dolores_Reyes_IPA\Informes\CERTIFICADO TAREAS.pdf
data\project\Dolores_Reyes_IPA\Informes\HOJA DE CITA.pdf
data\project\Dolores_Reyes_IPA\Informes\IMS.pdf
data\project\Dolores_Reyes_IPA\Informes\Informe 07.06.2021 ASISTENCIA ESPECIALIZADA.pdf
data\project\Dolores_Reyes_IPA\Informes\INFORME 12.08.2020.pdf
data\project\Dolores_Reyes_IPA\Informes\INFORME CONSULTA REVISIÓN.pdf
data\project\Dolores_Reyes_IPA\Informes\LOLA REYES - HOJA DE ANEMNESIS.pdf
data\project\Dolores_Reyes_IPA\Informes\Resolución INSS.pdf


In [7]:
question = """Instrucciones:
              1) Extrae de estos informes fechas de la asistencia seguido del centro de asistencia médica donde se atiende al paciente con un resumen de lo que pasó.
              2) Al principio de dicho resumen la siguiente frase "Informe de fecha". 
              3) Nunca mostrar nombres, solo el paciente o la paciente
              4) Escribirlo con vocabulario Médico. 
              5) No poner la edad. 
              6) Usar para la respuesta el mismo idioma que el nativo de los documentos.
"""

In [8]:
prompt =[f"""{question} """]+  content

In [9]:
response = get_chat_response(chat, prompt)

In [15]:
print(response)

## Extracción de información de informes médicos:

**Informe de fecha 24/06/2021 - Hospital San Juan de Dios del Aljarafe:** 

El paciente está citado para un estudio preanestésico previo a una intervención de traumatología. Se le indica que debe acudir en dos días diferentes: el primero para pruebas prequirúrgicas (analítica y electrocardiograma) y el segundo para la consulta de preanestesia. 

**Informe de fecha 11/06/2021 - INSS:**

El paciente, con profesión de teleoperador, se encuentra en situación de Incapacidad Temporal desde julio de 2019 debido a EPOC con obstrucción moderada, fractura no consolidada de clavícula, estenosis del canal medular L4-L5 y degenerativos L5-S1. Presenta además antecedentes de sífilis curada, VHC sin actividad actual, HTA y colon irritable. Actualmente refiere dolor por la fractura de clavícula no intervenida y reagudización de la ciática derecha. Se encuentra pendiente de pruebas de digestivo solicitadas en enero de 2021. 

**Informe de fecha 19/01/2

In [12]:
question = """organizar la informacion en una lista separada de puntos y da un resumen final"""
prompt=  [f"""{question} """]


In [13]:
response1 = get_chat_response(chat, prompt)
        

In [16]:
print(response1)

## Resumen de informes médicos:

* **24/06/2021:** Cita para estudio preanestésico previo a intervención de traumatología.
* **11/06/2021:**  Incapacidad Temporal desde julio de 2019 por EPOC, fractura de clavícula, estenosis del canal medular y degenerativos lumbares. Dolor por fractura de clavícula no intervenida y reagudización de ciática. Pendiente de pruebas digestivas.
* **19/01/2021:** RM lumbar informa espondilolistesis L4-L5 con estenosis del canal medular y cambios degenerativos.
* **15/04/2020:** Colonoscopia e ileoscopia normales. Diagnóstico de colon irritable y hemorroides internas. Se pautó tratamiento.
* **09/09/2021:**  Dolor lumbar y ciática derecha invalidante con claudicación de la marcha e incontinencia de esfínteres. Se plantea cirugía para espondilolistesis L4-L5.
* **11/11/2020:** Fractura no consolidada de clavícula izquierda. Se plantea tratamiento quirúrgico.

## Resumen final:

El paciente presenta una compleja situación médica con múltiples problemas de sal